In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
torch.manual_seed(42)


In [2]:
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('../Data/wiki-labeled.csv')
gpt = df[df['label'] == 0]
human = df[df['label'] == 1]
gpt = gpt.sample(n=150)
human = human.sample(n=150)

In [4]:
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:]
    return embeddings.numpy()


In [5]:
gpt['embeddings'] = gpt['text'].apply(generate_bert_embeddings)
human['embeddings'] = human['text'].apply(generate_bert_embeddings)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels, 8, kernel_size=16, stride=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=8, stride=2)

        self.conv2 = nn.Conv1d(8, 16, kernel_size=8, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=4, stride=2)

        # self.conv3 = nn.Conv2d(4, 128, kernel_size=3, stride=1, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(1920, 512)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(512, 128)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(128, 2)
        

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # x = self.pool3(self.relu3(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x) 

        return x


In [7]:
import torch
import torchvision.models as models

# Define the model architecture (make sure it matches the architecture of the saved model)
#model = models.resnet18()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(in_channels=1, num_classes=2).to(device)
# Path to the saved model
model_path = "/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/model/wiki_model_1D.pth"

# Load the saved model parameters into the model
model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
model.eval()

# Now the model is loaded and ready for inference or further training


CNN(
  (conv1): Conv1d(1, 8, kernel_size=(16,), stride=(2,))
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=8, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(8, 16, kernel_size=(8,), stride=(1,))
  (relu2): ReLU()
  (pool2): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1920, out_features=512, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (relu5): ReLU()
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [9]:
gpt_embeds = torch.Tensor(gpt['embeddings'].tolist())
human_embeds = torch.Tensor(human['embeddings'].tolist())

In [15]:
gpt_pred = []
for embed in gpt_embeds:
    embed = embed.unsqueeze(0).to(device)
    #embed = embed.unsqueeze(0)
    output = model(embed)
    _, predicted = torch.max(output.data, 1)
    #print(predicted.item())
    gpt_pred.append(predicted.item())

In [16]:
from collections import Counter
print(Counter(gpt_pred))

Counter({1: 150})


In [12]:
human_pred = []
for embed in human_embeds:
    embed = embed.unsqueeze(0).to(device)
    #embed = embed.unsqueeze(0)
    output = model(embed)
    _, predicted = torch.max(output.data, 1)
    #print(predicted.item())
    human_pred.append(predicted.item())

In [13]:
print(Counter(human_pred))

Counter({1: 150})
